In [ ]:
filename = 'input.txt'

with open(filename) as f:
    maze = [list(line.strip()) for line in f.readlines()]

start = [0,0]
end = [0,0]

for y, row in enumerate(maze):
    for x, elem in enumerate(row):
        if elem == 'E':
            end = [y, x]
        elif elem == 'S':
            start = [y, x]

memo = {}

In [ ]:
# a star is common goods, so I got mine from here: https://www.geeksforgeeks.org/a-search-algorithm/

# Python program for A* Search Algorithm
import math
import heapq

# Define the Cell class

class Cell:
    def __init__(self):
      # Parent cell's row index
        self.parent_i = 0
    # Parent cell's column index
        self.parent_j = 0
 # Total cost of the cell (g + h)
        self.f = float('inf')
    # Cost from start to this cell
        self.g = float('inf')
    # Heuristic cost from this cell to destination
        self.h = 0
    # distance to goal if found
        self.dist_to_goal = None

    def __str__(self):
        if self.dist_to_goal:
            return str(self.dist_to_goal)
        elif self.f != float('inf'):
            return 'X'
        else:
            return "."


# Define the size of the grid
ROW = len(maze)
COL = len(maze[0])

# Check if a cell is valid (within the grid)


def is_valid(row, col):
    return (row >= 0) and (row < ROW) and (col >= 0) and (col < COL)

# Check if a cell is unblocked


def is_unblocked(grid, row, col):
    return grid[row][col] != '#'

# Check if a cell is the destination


def is_destination(row, col, dest):
    return row == dest[0] and col == dest[1]

# Calculate the heuristic value of a cell (Euclidean distance to destination)


def calculate_h_value(row, col, dest):
    return ((row - dest[0]) ** 2 + (col - dest[1]) ** 2) ** 0.5

# Trace the path from source to destination


def trace_path(cell_details, dest):
    # print("The Path is ")
    global memo_cells
    path = []
    row = dest[0]
    col = dest[1]

    # Trace the path from destination to source using parent cells
    while not (cell_details[row][col].parent_i == row and cell_details[row][col].parent_j == col):
        path.append((row, col))
        temp_row = cell_details[row][col].parent_i
        temp_col = cell_details[row][col].parent_j
        row = temp_row
        col = temp_col

    # Add the source cell to the path
    path.append((row, col))
    # since we also want to use the reverse path, we make a copy
    reverse_path = path[:]
    # Reverse the path to get the path from source to destination
    path.reverse()

    for y, row in enumerate(cell_details):
        for x, elem in enumerate(row):
            if elem.f != float('inf'):
                memo_cells[y][x].parent_i = elem.parent_i
                memo_cells[y][x].parent_j = elem.parent_j

    for i, elem in enumerate(reverse_path):
        memo_cells[elem[0]][elem[1]].dist_to_goal = i

    # Print the path
    # for i in path:
    #    print("->", i, end=" ")
    # print()
    # print("length", len(path)-1)
    return len(path) - 1, reverse_path

memo_cells = cell_details = [[Cell() for _ in range(COL)] for _ in range(ROW)]

def handle_found_memo(y, x, cell_details):
    global memo_cells
    # print("using memo")
    row = y
    col = x

    path = []

    # Trace the path from destination to source using parent cells
    while not (cell_details[row][col].parent_i == row and cell_details[row][col].parent_j == col):
        path.append((row, col))
        temp_row = cell_details[row][col].parent_i
        temp_col = cell_details[row][col].parent_j
        row = temp_row
        col = temp_col

    if memo_cells[y][x].dist_to_goal is not None:
        for i, elem in enumerate(path):
            if i == 0:
                continue
            memo_cells[elem[0]][elem[i]].dist_to_goal = memo_cells[y][x].dist_to_goal + i
        return len(path) - 1 + memo_cells[y][x].dist_to_goal

    else:
        row = y
        col = x
        while not cell_details[row][col].dist_to_goal:
            path.append((row, col))
            temp_row = cell_details[row][col].parent_i
            temp_col = cell_details[row][col].parent_j
            row = temp_row
            col = temp_col
        # now pos[y][x] is twice in the list
        return len(path) - 2 + memo_cells[row][col].dist_to_goal



# Implement the A* search algorithm
def a_star_search(grid, src, dest):
    global memo
    # Check if the source and destination are valid
    if not is_valid(src[0], src[1]) or not is_valid(dest[0], dest[1]):
        print("Source or destination is invalid")
        return

    # Check if the source and destination are unblocked
    if not is_unblocked(grid, src[0], src[1]) or not is_unblocked(grid, dest[0], dest[1]):
        print("Source or the destination is blocked")
        return

    # Check if we are already at the destination
    if is_destination(src[0], src[1], dest):
        print("We are already at the destination")
        return

    # Initialize the closed list (visited cells)
    closed_list = [[False for _ in range(COL)] for _ in range(ROW)]
    # Initialize the details of each cell
    cell_details = [[Cell() for _ in range(COL)] for _ in range(ROW)]

    # Initialize the start cell details
    i = src[0]
    j = src[1]
    cell_details[i][j].f = 0
    cell_details[i][j].g = 0
    cell_details[i][j].h = 0
    cell_details[i][j].parent_i = i
    cell_details[i][j].parent_j = j

    # Initialize the open list (cells to be visited) with the start cell
    open_list = []
    heapq.heappush(open_list, (0.0, i, j))

    # Initialize the flag for whether destination is found
    found_dest = False

    # Main loop of A* search algorithm
    while len(open_list) > 0:
        # Pop the cell with the smallest f value from the open list
        p = heapq.heappop(open_list)

        # Mark the cell as visited
        i = p[1]
        j = p[2]
        closed_list[i][j] = True

        global memo_cells
        if memo_cells[i][j].f != float('inf') and memo_cells[i][j].dist_to_goal:
            handle_found_memo(i, j, cell_details)


        # For each direction, check the successors
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        for dir in directions:
            new_i = i + dir[0]
            new_j = j + dir[1]

            # If the successor is valid, unblocked, and not visited
            if is_valid(new_i, new_j) and is_unblocked(grid, new_i, new_j) and not closed_list[new_i][new_j]:
                # If the successor is the destination
                if is_destination(new_i, new_j, dest):
                    # Set the parent of the destination cell
                    cell_details[new_i][new_j].parent_i = i
                    cell_details[new_i][new_j].parent_j = j
                    # print("The destination cell is found")
                    # Trace and print the path from source to destination
                    return trace_path(cell_details, dest)
                    # found_dest = True
                    # return
                else:
                    # Calculate the new f, g, and h values
                    g_new = cell_details[i][j].g + 1.0
                    h_new = calculate_h_value(new_i, new_j, dest)
                    f_new = g_new + h_new

                    # If the cell is not in the open list or the new f value is smaller
                    if cell_details[new_i][new_j].f == float('inf') or cell_details[new_i][new_j].f > f_new:
                        # Add the cell to the open list
                        heapq.heappush(open_list, (f_new, new_i, new_j))
                        # Update the cell details
                        cell_details[new_i][new_j].f = f_new
                        cell_details[new_i][new_j].g = g_new
                        cell_details[new_i][new_j].h = h_new
                        cell_details[new_i][new_j].parent_i = i
                        cell_details[new_i][new_j].parent_j = j

    # If the destination is not found after visiting all cells
    if not found_dest:
        print("Failed to find the destination cell")

# Driver Code


def main(maze_version, start_pos, end_pos):

    # Define the source and destination
    # src = [0, 0]
    # dest = [6, 6]

    # Run the A* search algorithm
    return a_star_search(maze_version, start_pos, end_pos)


# print(main(maze, start, end))
total_length, reverse_path = main(maze, start, end)

In [ ]:
# from start is len - 1 - index in reverse
total_length

In [ ]:
directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def is_valid_starting_point(position, path_idx):
    if (0 <= position[0] < len(maze) and 0 <= position[1] < len(maze[0])
            and maze[position[0]][position[1]] in ['E','.'] and position not in [reverse_path[path_idx-2], reverse_path[path_idx+2]]):
        return True
    return False

win_count = 0
biggest_win = 0

for index, elem in enumerate(reverse_path):
    # if index % 1000 == 0:
    #    print(index)
    # if index != 72:
    #    continue
    if index < 2 or index > len(reverse_path) - 3:
        continue
    new_starts = []

    for dir in directions:
        cheated_position = (elem[0] + 2 * dir[0], elem[1] + 2 * dir[1])
        if is_valid_starting_point(cheated_position, index):
            new_starts.append(cheated_position)

    for start in new_starts:
        # print(main(maze, start, end))
        if start == (end[0], end[1]):
            end_length = 0
        else:
            end_length, _ = main(maze, start, end)
        # so end length + what was traversed by total length + 2 for the cheat
        cheat_length = end_length + (total_length - index) + 2
        # print(cheat_length)
        win = total_length - cheat_length
        if win >= 100:
            win_count += 1
            if win > biggest_win:
                biggest_win = win
        # print("win", total_length - cheat_length)


In [ ]:
win_count

In [ ]:
biggest_win

In [ ]:
for row in memo_cells:
    print([str(x) for x in row])